In [3]:
from bs4 import BeautifulSoup #for scraping
import requests               #required for reading the file
import pandas as pd           #(optional) Pandas for dataframes 
import json                   #(optional) If you want to export json
import os

In [5]:
url = input('Enter Youtube Video Url- ') # user input for the link
vid={}
link = url
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')
div_s = soup.find_all('div')
title = div_s[1].find('span',class_='watch-title').text.strip()
vid['Title']= title
vid['Link']= link
channel_name = div_s[1].find('a',class_="yt-uix-sessionlink spf-link").text.strip()
channel_link = ('www.youtube.com'+div_s[1].find('a',class_="yt-uix-sessionlink spf-link").get('href'))
subscribers = div_s[1].find('span',class_="yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count").text.strip()
if len(channel_name) == 0:
    channel_name ='None'
    channel_link = 'None'
    subscribers = 'None'
vid['Channel']= channel_name
vid['Channel_link']= channel_link
vid['Channel_subscribers']= cubscribers

Enter Youtube Video Url- https://www.youtube.com/watch?v=9JUAPgtkKpI


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
class YTstats:
    def __init__(self,api_key,channel_id):
        self.api_key = api_key
        self.channel_id = channel_id
        self.channel_statistics = None
        self.video_data = None 
        
    def get_channel_statistics(self):
        url = f'https://www.googleapis.com/youtube/v3/channels?part=statistics&id={self.channel_id}&key={self.api_key}'
#         print(url)
        json_url = requests.get(url)
        print(json_url.status_code)
#         print(json_url.text)
        data =json.loads(json_url.text)
#         print(data)
        try:
            data = data["items"][0]["statistics"]
        except:
            data = None 
        self.channel_statistics = data 
        return data
    
    def get_channel_video_data(self):
        channel_videos = self._get_channel_video(limit = 50)
        print(channel_videos)
        parts = ["snippet","statistics","contentDetails"]
        for video_id in channel_videos:
            for part in parts:
                data = self._get_single_video_data(video_id, part)
                channel_videos[video_id].update(data)
            
            self.video_data = channel_videos
            return channel_videos 
                
    def _get_single_video_data(self,video_id,part):
        url = f"https://www.googleapis.com/youtube/v3/videos?part={part}&id={video_id}&key={self.api_key}"
        json_url = requests.get(url)
        data = json.loads(json_url.text)
        try: 
            data = data['items'][0][part]
        except:
            print('Oh no!')
            data = dict()
            
        return data    
        
    def _get_channel_video(self, limit= None):
        url = f'https://www.googleapis.com/youtube/v3/search?key={self.api_key}&channelId={self.channel_id}&part=id&order=date'
        if limit is not None and isinstance(limit,int):
            url += "&maxResults=" + str(limit)
        
        vid,npt = self._get_channel_video_per_page(url)
        indx = 0
        while(npt is not None and indx <10):
            nexturl = url + "&PageToken=" + npt
            next_vid,npt = self._get_channel_video_per_page(nexturl)
            vid.update(next_vid)
            indx +=1
        return vid
       
   
    def _get_channel_video_per_page(self,url):
        json_url = requests.get(url)
        data = json.loads(json_url.text)
        channel_videos = dict()
        if 'items' not in data:
            return channel_videos, None
        
        item_data = data['items']
        nextPageToken = data.get("nextPageToken",None)
        for item in item_data:
            try:
                kind = item['id']['kind']
                if kind == 'youtube#video':
                    video_id = item['id']['videoId']
                    channel_videos[video_id] = dict()
            except KeyError:
                print("Oh no!")
                
        return channel_videos, nextPageToken 
        
    def dump(self):
        if self.channel_statistics is None or self.video_data is None:
            print('data is none')
            return 
        
        combined_data = {self.channel_id:{"channel_statistics":self.channel_statistics,"video_data":self.video_data}}
        
        channel_title = self.video_data.popitem()[1].get('channelTitle', self.channel_id)
        channel_title = channel_title.replace(" ","_").lower()
        file_name = channel_title + '.json'
        with open(file_name, 'w') as f:
            json.dump(combined_data, f, indent = 4)
        
        print('file dumped') 